In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.optimize import least_squares
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Input, Dense, SimpleRNN, Dropout, LSTM, GRU, BatchNormalization, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from keras.regularizers import l2


In [33]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [28]:
# Read the CSV file and sample 40% of the data
data = pd.read_csv('flight_data_2021.csv')
data_sampled = data.sample(frac=0.4, random_state=42) # 40%

In [4]:
data_sampled.to_csv('sample.csv',index=False)

In [29]:
columns_to_drop = ['Origin','Dest','DestState','OriginState','DepDelayMinutes',
                   'Operated_or_Branded_Code_Share_Partners',
                   'arr-type','arr-elevation_ft']
clean_data = data.drop(columns_to_drop, axis=1)
clean_data.head()

,Airline,Cancelled,Quarter,Month,DayOfWeek,DepTimeBlk,dept-type,dept-elevation_ft,Route_Popularity,Distance_Final,Wind_Speed_mph,Wind_Gust_mph,Visibility_miles,tempF,precip_in,daily_snow_in
0,SkyWest Airlines Inc.,False,1,3,3,0700-0759,medium_airport,2941.0,937.0,422.246813,10.0,17.0,6.0,49,0.0,0.0
1,SkyWest Airlines Inc.,False,1,3,3,0900-0959,large_airport,1135.0,937.0,422.246813,6.0,9.0,6.0,60,0.0,0.0
2,SkyWest Airlines Inc.,False,1,3,3,1300-1359,large_airport,266.0,918.0,1356.657710,10.0,18.0,6.0,39,0.0,0.0
3,SkyWest Airlines Inc.,False,1,3,3,1600-1659,large_airport,607.0,568.0,1396.572313,10.0,16.0,6.0,65,0.0,0.0
4,SkyWest Airlines Inc.,False,1,3,3,1800-1859,large_airport,1135.0,1135.0,684.252343,21.0,35.0,6.0,71,0.0,0.0


## Undersampling Major Class

In [22]:
# Applied Downsampling to balance the weight for minor class "Cancelled"
data_majority = clean_data[clean_data['Cancelled'] == False]
data_minority = clean_data[clean_data['Cancelled'] == True]

# Step 3: Downsample the majority class
data_majority_downsampled = resample(data_majority, replace=False, n_samples=len(data_minority), random_state=123)

# Step 4: Combine minority class with downsampled majority class
data_downsampled = pd.concat([data_majority_downsampled, data_minority])

# Display new class counts
print(data_downsampled['Cancelled'].value_counts())

Cancelled
False    106876
True     106876
Name: count, dtype: int64


In [23]:
# Assuming data_downsampled is your DataFrame with features and target
# Separate features and target
X = data_downsampled.drop(columns=['Cancelled'])
y = data_downsampled['Cancelled']

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(include=['number']).columns

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Preprocess the data
X_processed = preprocessor.fit_transform(X)

## Simple RNN

In [17]:
# Reshape for LSTM [samples, timesteps, features]
# Here, we assume each sample is treated as one timestep with all features
# X_processed = X_processed.reshape((X_processed.shape[0], 1, X_processed.shape[1]))

# Split the data
# X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Split data into train, validation, and test sets
X_train, X_temp1, y_train, y_temp1 = train_test_split(X_processed, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp1, y_temp1, test_size=0.5, random_state=42)

# Build the RNN model 1
model = Sequential()
model.add(SimpleRNN(50, activation='relu', return_sequences=True, 
                    input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(SimpleRNN(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Save the model
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32,
                    validation_data=(X_val, y_val),
                    verbose=2)

# Save the model
model.save('simple_rnn.h5')

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Test Accuracy: {accuracy:.4f}')

# Predicting
y_pred = model.predict(X_test)
y_pred_classes = np.where(y_pred > 0.5, 1, 0)

# Print classification report or confusion matrix
print(classification_report(y_test, y_pred_classes))
print(confusion_matrix(y_test, y_pred_classes))


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_4 (SimpleRNN)    (None, 1, 50)             5300      
                                                                 
 dropout_10 (Dropout)        (None, 1, 50)             0         
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 50)                5050      
                                                                 
 dropout_11 (Dropout)        (None, 50)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                 51        
                                                                 
Total params: 10,401
Trainable params: 10,401
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
4676/4676 - 42s - loss: 0.6082 - accura

## Change the structure of RNN (Include LSTM and GRU) - Hybrid RNN

In [18]:
# Build the RNN model 2
model = Sequential()

# First RNN layer (LSTM)
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.3))
model.add(BatchNormalization())

# Second RNN layer (GRU)
model.add(GRU(100, activation='relu', return_sequences=True))
model.add(Dropout(0.3))
model.add(BatchNormalization())

# Third RNN layer (LSTM)
model.add(LSTM(50, activation='relu', return_sequences=False))
model.add(Dropout(0.3))
model.add(BatchNormalization())

# Dense layer
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.3))

# Output layer
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=30, 
                    batch_size=32, 
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping, reduce_lr])

# Save the model
model.save('hybrid.h5')

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Test Accuracy: {accuracy:.4f}')

# Predicting
y_pred = model.predict(X_test)
y_pred_classes = np.where(y_pred > 0.5, 1, 0)

# Print classification report or confusion matrix
print(classification_report(y_test, y_pred_classes))
print(confusion_matrix(y_test, y_pred_classes))


Epoch 1/30
4676/4676 [==============================] - 82s 16ms/step - loss: 0.6256 - accuracy: 0.6412 - val_loss: 0.5760 - val_accuracy: 0.6855 - lr: 0.0010
Epoch 2/30
4676/4676 [==============================] - 75s 16ms/step - loss: 0.5882 - accuracy: 0.6824 - val_loss: 0.5552 - val_accuracy: 0.7076 - lr: 0.0010
Epoch 3/30
4676/4676 [==============================] - 75s 16ms/step - loss: 0.5719 - accuracy: 0.6985 - val_loss: 0.5372 - val_accuracy: 0.7241 - lr: 0.0010
Epoch 4/30
4676/4676 [==============================] - 75s 16ms/step - loss: 0.5605 - accuracy: 0.7071 - val_loss: 0.5291 - val_accuracy: 0.7318 - lr: 0.0010
Epoch 5/30
4676/4676 [==============================] - 75s 16ms/step - loss: 0.5530 - accuracy: 0.7149 - val_loss: 0.5208 - val_accuracy: 0.7332 - lr: 0.0010
Epoch 6/30
4676/4676 [==============================] - 75s 16ms/step - loss: 0.5468 - accuracy: 0.7173 - val_loss: 0.5186 - val_accuracy: 0.7362 - lr: 0.0010
Epoch 7/30
4676/4676 [========================

## Stacked LSTM

In [14]:
# Define the enhanced stacked LSTM model
model = Sequential()
model.add(Bidirectional(LSTM(64, activation='relu', return_sequences=True, 
                             kernel_regularizer=l2(0.01)), input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(64, activation='relu', return_sequences=True, 
                             kernel_regularizer=l2(0.01))))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(LSTM(32, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
model_checkpoint = ModelCheckpoint(filepath='best_model_stacked.h5', monitor='val_loss', 
                                   save_best_only=True, verbose=1)

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=30, 
                    batch_size=32, 
                    validation_data=(X_val, y_val), 
                    callbacks=[early_stopping, reduce_lr,model_checkpoint])

# Load the best model
best_model = load_model('best_model_stacked.h5')

# Evaluate the model
loss, accuracy = best_model.evaluate(X_test, y_test, verbose=2)
print(f'Test Accuracy: {accuracy:.4f}')

# Predicting
y_pred = best_model.predict(X_test)
y_pred_classes = np.where(y_pred > 0.5, 1, 0)

# Print classification report or confusion matrix
print(classification_report(y_test, y_pred_classes))
print(confusion_matrix(y_test, y_pred_classes))

Epoch 1/30
4676/4676 [==============================] - ETA: 0s - loss: 0.8505 - accuracy: 0.6389
Epoch 1: val_loss improved from inf to 0.62716, saving model to best_model_stacked.h5
4676/4676 [==============================] - 149s 29ms/step - loss: 0.8505 - accuracy: 0.6389 - val_loss: 0.6272 - val_accuracy: 0.6706 - lr: 0.0010
Epoch 2/30
4676/4676 [==============================] - ETA: 0s - loss: 0.6394 - accuracy: 0.6578
Epoch 2: val_loss improved from 0.62716 to 0.62070, saving model to best_model_stacked.h5
4676/4676 [==============================] - 136s 29ms/step - loss: 0.6394 - accuracy: 0.6578 - val_loss: 0.6207 - val_accuracy: 0.6681 - lr: 0.0010
Epoch 3/30
4676/4676 [==============================] - ETA: 0s - loss: 0.6337 - accuracy: 0.6586
Epoch 3: val_loss improved from 0.62070 to 0.60969, saving model to best_model_stacked.h5
4676/4676 [==============================] - 137s 29ms/step - loss: 0.6337 - accuracy: 0.6586 - val_loss: 0.6097 - val_accuracy: 0.6773 - lr: 

### SDA-LSTM (previously SDA-LM) Method

https://journalofbigdata.springeropen.com/articles/10.1186/s40537-020-00380-z

In [44]:
# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(include=['number']).columns

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Bundle preprocessing for numerical and categorical data
preprocessor2 = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Preprocess the data
X_processed2 = preprocessor2.fit_transform(X)

# Add the target column back
data = np.hstack((X_processed2, y.values.reshape(-1, 1)))

# Convert data to sequences
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, :-1])
        y.append(data[i+seq_length, -1])
    return np.array(X), np.array(y)

seq_length = 10  # Example sequence length
X2, y2 = create_sequences(data, seq_length)

# Split data into train, validation, and test sets
X_train2, X_temp, y_train2, y_temp = train_test_split(X2, y2, test_size=0.3, random_state=42)
X_val2, X_test2, y_val2, y_test2 = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Define denoising autoencoder with separate encoder and decoder
def build_autoencoder(input_dim, encoding_dim):
    # Encoder
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(encoding_dim, activation='relu')(input_layer)
    encoder = Model(input_layer, encoded)

    # Decoder
    encoded_input = Input(shape=(encoding_dim,))
    decoded = Dense(input_dim, activation='sigmoid')(encoded_input)
    decoder = Model(encoded_input, decoded)

    # Autoencoder
    autoencoder = Model(input_layer, decoder(encoder(input_layer)))

    return autoencoder, encoder, decoder

input_dim = X_train2.shape[2]  # Number of features
encoding_dim_1 = 64  # First encoding dimension
encoding_dim_2 = 32  # Second encoding dimension

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Build first autoencoder
autoencoder1, encoder1, decoder1 = build_autoencoder(input_dim, encoding_dim_1)
autoencoder1.compile(optimizer='adam', loss='mse')
autoencoder1.fit(X_train2.reshape(-1, input_dim), 
                 X_train2.reshape(-1, input_dim), 
                 epochs=10, 
                 batch_size=64, 
                 shuffle=True, 
                 validation_split=0.2, 
                 callbacks=[early_stopping])

# Encode the data
encoded_train1 = encoder1.predict(X_train2.reshape(-1, input_dim))
encoded_val1 = encoder1.predict(X_val2.reshape(-1, input_dim))
encoded_test1 = encoder1.predict(X_test2.reshape(-1, input_dim))

# Build second autoencoder
autoencoder2, encoder2, decoder2 = build_autoencoder(encoding_dim_1, encoding_dim_2)
autoencoder2.compile(optimizer='adam', loss='mse')
autoencoder2.fit(encoded_train1, 
                 encoded_train1, 
                 epochs=10, 
                 batch_size=64, 
                 shuffle=True, 
                 validation_split=0.2, 
                 callbacks=[early_stopping])

# Encode the data again
encoded_train2 = encoder2.predict(encoded_train1).reshape(X_train2.shape[0], seq_length, encoding_dim_2)
encoded_val2 = encoder2.predict(encoded_val1).reshape(X_val2.shape[0], seq_length, encoding_dim_2)
encoded_test2 = encoder2.predict(encoded_test1).reshape(X_test2.shape[0], seq_length, encoding_dim_2)

# Build and compile the final model with LSTM layers
model = Sequential()
model.add(LSTM(64, activation='relu', return_sequences=True, input_shape=(seq_length, encoding_dim_2)))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(LSTM(32, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
model_checkpoint = ModelCheckpoint(filepath='best_model_sda_lm.h5', monitor='val_loss', 
                                   save_best_only=True, verbose=1)

# Train the model
history = model.fit(encoded_train2, y_train2, 
                    epochs=2, 
                    batch_size=64, 
                    validation_data=(encoded_val2, y_val2), 
                    callbacks=[early_stopping, reduce_lr, model_checkpoint])

# Save the autoencoders
autoencoder1.save('autoencoder1.h5')
autoencoder2.save('autoencoder2.h5')

Epoch 1/10
18703/18703 [==============================] - 63s 3ms/step - loss: 0.0020 - val_loss: 2.2096e-04
Epoch 2/10
18703/18703 [==============================] - 62s 3ms/step - loss: 2.1477e-04 - val_loss: 2.0239e-04
Epoch 3/10
18703/18703 [==============================] - 61s 3ms/step - loss: 1.9627e-04 - val_loss: 1.8808e-04
Epoch 4/10
18703/18703 [==============================] - 61s 3ms/step - loss: 1.8353e-04 - val_loss: 1.8212e-04
Epoch 5/10
18703/18703 [==============================] - 62s 3ms/step - loss: 1.7186e-04 - val_loss: 1.5436e-04
Epoch 6/10
18703/18703 [==============================] - 63s 3ms/step - loss: 1.4173e-04 - val_loss: 1.2803e-04
Epoch 7/10
18703/18703 [==============================] - 62s 3ms/step - loss: 1.1710e-04 - val_loss: 1.1406e-04
Epoch 8/10
18703/18703 [==============================] - 62s 3ms/step - loss: 1.0727e-04 - val_loss: 1.0566e-04
Epoch 9/10
18703/18703 [==============================] - 60s 3ms/step - loss: 1.0435e-04 - val_loss

In [ ]:
# Load the best model
best_model = load_model('best_model_sda_lm.h5')

# Evaluate the best model
loss, accuracy = best_model.evaluate(encoded_test2, y_test2)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

# Classification report for the best model
y_pred = (best_model.predict(encoded_test2) > 0.5).astype("int32")
print(confusion_matrix(y_test2, y_pred))

## Apply the same methodology in the Imbalanced Dataset?

In [45]:
X2 = clean_data.drop(columns='Cancelled',axis=1)
y2 = clean_data['Cancelled']

# Identify categorical and numerical columns
categorical_cols = X2.select_dtypes(include=['object']).columns
numerical_cols = X2.select_dtypes(include=['number']).columns

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Bundle preprocessing for numerical and categorical data
preprocessor2_imb = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Preprocess the data
X_processed2_imb = preprocessor2_imb.fit_transform(X2)

# Add the target column back
data2 = np.hstack((X_processed2, y2.values.reshape(-1, 1)))

seq_length = 10  # Example sequence length
X2, y2 = create_sequences(data2, seq_length)

# Split data into train, validation, and test sets
X_train2, X_temp, y_train2, y_temp = train_test_split(X2, y2, test_size=0.3, random_state=42)
X_val2, X_test2, y_val2, y_test2 = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

input_dim = X_train2.shape[2]  # Number of features
encoding_dim_1 = 64  # First encoding dimension
encoding_dim_2 = 32  # Second encoding dimension

# Build first autoencoder
autoencoder1, encoder1, decoder1 = build_autoencoder(input_dim, encoding_dim_1)
autoencoder1.compile(optimizer='adam', loss='mse')
autoencoder1.fit(X_train2.reshape(-1, input_dim), 
                 X_train2.reshape(-1, input_dim), 
                 epochs=10, 
                 batch_size=64, 
                 shuffle=True, 
                 validation_split=0.2, 
                 callbacks=[early_stopping])

# Encode the data
encoded_train1 = encoder1.predict(X_train2.reshape(-1, input_dim))
encoded_val1 = encoder1.predict(X_val2.reshape(-1, input_dim))
encoded_test1 = encoder1.predict(X_test2.reshape(-1, input_dim))

# Build second autoencoder
autoencoder2, encoder2, decoder2 = build_autoencoder(encoding_dim_1, encoding_dim_2)
autoencoder2.compile(optimizer='adam', loss='mse')
autoencoder2.fit(encoded_train1, 
                 encoded_train1, 
                 epochs=10, 
                 batch_size=64, 
                 shuffle=True, 
                 validation_split=0.2, 
                 callbacks=[early_stopping])

# Encode the data again
encoded_train2 = encoder2.predict(encoded_train1).reshape(X_train2.shape[0], seq_length, encoding_dim_2)
encoded_val2 = encoder2.predict(encoded_val1).reshape(X_val2.shape[0], seq_length, encoding_dim_2)
encoded_test2 = encoder2.predict(encoded_test1).reshape(X_test2.shape[0], seq_length, encoding_dim_2)

# Build and compile the final model with LSTM layers
model = Sequential()
model.add(LSTM(64, activation='relu', return_sequences=True, input_shape=(seq_length, encoding_dim_2)))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(LSTM(32, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
model_checkpoint = ModelCheckpoint(filepath='best_model_sda_lm.h5', monitor='val_loss', 
                                   save_best_only=True, verbose=1)

# Train the model
history = model.fit(encoded_train2, y_train2, 
                    epochs=2, 
                    batch_size=64, 
                    validation_data=(encoded_val2, y_val2), 
                    callbacks=[early_stopping, reduce_lr, model_checkpoint])

# Save the autoencoders
autoencoder1.save('autoencoder1.h5')
autoencoder2.save('autoencoder2.h5')

Epoch 1/10
18703/18703 [==============================] - 59s 3ms/step - loss: 0.0019 - val_loss: 2.2277e-04
Epoch 2/10
18703/18703 [==============================] - 61s 3ms/step - loss: 2.1176e-04 - val_loss: 2.0681e-04
Epoch 3/10
18703/18703 [==============================] - 61s 3ms/step - loss: 2.0659e-04 - val_loss: 2.0255e-04
Epoch 4/10
18703/18703 [==============================] - 60s 3ms/step - loss: 1.7760e-04 - val_loss: 1.4747e-04
Epoch 5/10
18703/18703 [==============================] - 62s 3ms/step - loss: 1.4579e-04 - val_loss: 1.4546e-04
Epoch 6/10
18703/18703 [==============================] - 59s 3ms/step - loss: 1.4301e-04 - val_loss: 1.2332e-04
Epoch 7/10
18703/18703 [==============================] - 57s 3ms/step - loss: 1.1487e-04 - val_loss: 1.0905e-04
Epoch 8/10
18703/18703 [==============================] - 59s 3ms/step - loss: 1.0616e-04 - val_loss: 1.0687e-04
Epoch 9/10
18703/18703 [==============================] - 59s 3ms/step - loss: 1.0391e-04 - val_loss

ValueError: in user code:

    File "/nfs/home/mnk1906/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1852, in test_function  *
        return step_function(self, iterator)
    File "/nfs/home/mnk1906/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1836, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/nfs/home/mnk1906/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1824, in run_step  **
        outputs = model.test_step(data)
    File "/nfs/home/mnk1906/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1788, in test_step
        y_pred = self(x, training=False)
    File "/nfs/home/mnk1906/.local/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/nfs/home/mnk1906/.local/lib/python3.9/site-packages/keras/engine/input_spec.py", line 298, in assert_input_compati

In [48]:
# Load the best model
best_model = load_model('best_model_sda_lm.h5')

# Evaluate the best model
loss, accuracy = best_model.evaluate(encoded_test2, y_test2)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

# Classification report for the best model
y_pred = (best_model.predict(encoded_test2) > 0.5).astype("int32")
print(confusion_matrix(y_test2, y_pred))
print(classification_report(y_test2, y_pred))

1002/1002 [==============================] - 13s 12ms/step - loss: 0.0042 - accuracy: 0.9988
Test Loss: 0.004196505062282085, Test Accuracy: 0.9988459944725037
1002/1002 [==============================] - 11s 11ms/step
[[15877     1]
 [   36 16148]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     15878
         1.0       1.00      1.00      1.00     16184

    accuracy                           1.00     32062
   macro avg       1.00      1.00      1.00     32062
weighted avg       1.00      1.00      1.00     32062

